In [1]:
from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn import linear_model, preprocessing
import statsmodels.formula.api as smf
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn import datasets, linear_model
import math
import scipy
import warnings
import sklearn
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import random
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format
warnings.filterwarnings('ignore')

In [2]:
df13 = pd.read_csv('https://raw.githubusercontent.com/Thinkful-Ed/data-201-resources/master/New_York_offenses/NEW_YORK-Offenses_Known_to_Law_Enforcement_by_City_2013%20-%2013tbl8ny.csv', header=4, skipfooter=3)
df14 = pd.read_csv('offenses_14.csv', header=4, skipfooter=3)

df13.columns = ['city', 'population', 'violent_crime', 'murder','rape(new def)', 'rape', 'robbery', 'aggrevated_assault', 'propertycrime', 'burglary', 'larceny', 'motor_theft', 'arson']
df14.columns = ['city', 'population', 'violent_crime', 'murder','rape(new def)', 'rape', 'robbery', 'aggrevated_assault', 'propertycrime', 'burglary', 'larceny', 'motor_theft', 'arson']

df13 = df13.drop(columns=['city','rape(new def)', 'arson']).replace(',', '', regex=True).dropna().astype(float)
df14 = df14.drop(columns=['city','rape(new def)', 'arson']).replace(',', '', regex=True).fillna(0).astype(float)

df13['population_sq'] = df13['population']**2
df14['population_sq'] = df14['population']**2

df13 = df13[['population', 'population_sq', 'murder', 'robbery', 'propertycrime']]
df14 = df14[['population', 'population_sq', 'murder', 'robbery', 'propertycrime']]

df14 = df14.sample(frac=.933)

In [3]:
p_outlier = df13['population'].median() + 2 * df13['population'].std()
m_outlier = df13['murder'].median() + 2 * df13['murder'].std()
r_outlier = df13['robbery'].median() + 2 * df13['robbery'].std()
pc_outlier = df13['propertycrime'].median() + 2 * df13['propertycrime'].std()

In [4]:
p_outlier = df14['population'].median() + 2 * df14['population'].std()
m_outlier = df14['murder'].median() + 2 * df14['murder'].std()
r_outlier = df14['robbery'].median() + 2 * df14['robbery'].std()
pc_outlier = df14['propertycrime'].median() + 2 * df14['propertycrime'].std()

In [5]:
df13['population'] = df13['population'].map(lambda x: x if x < p_outlier else 900000)
df13['murder'] = df13['murder'].map(lambda x: x if x < m_outlier else 36)
df13['robbery'] = df13['robbery'].map(lambda x: x if x < r_outlier else 2000)
df13['propertycrime'] = df13['propertycrime'].map(lambda x: x if x < pc_outlier else 15000)
df13['murder_bl'] = np.where(df13['murder'] > 0, 1, 0)
df13['robbery_bl'] = np.where(df13['robbery'] > 0, 1, 0)
df13 = df13[['population', 'population_sq', 'murder','robbery', 'propertycrime', 'murder_bl', 'robbery_bl']]

In [6]:
df14['population'] = df14['population'].map(lambda x: x if x < p_outlier else 900000)
df14['murder'] = df14['murder'].map(lambda x: x if x < m_outlier else 36)
df14['robbery'] = df14['robbery'].map(lambda x: x if x < r_outlier else 2000)
df14['propertycrime'] = df14['propertycrime'].map(lambda x: x if x < pc_outlier else 15000)
df14['murder_bl'] = np.where(df14['murder'] > 0, 1, 0)
df14['robbery_bl'] = np.where(df14['robbery'] > 0, 1, 0)
df14 = df14[['population', 'population_sq', 'murder','robbery', 'propertycrime', 'murder_bl', 'robbery_bl']]

In [7]:
regr1 = linear_model.LinearRegression()
y1 = df13[['propertycrime']]
X1 = df13[['population', 'population_sq', 'murder_bl', 'robbery_bl']]
regr1 = regr1.fit(X1, y1)

In [8]:
regr2 = linear_model.LinearRegression()
y2 = df14[['propertycrime']]
X2 = df14[['population', 'population_sq', 'murder_bl', 'robbery_bl']]
regr2 = regr2.fit(X2, y2)

In [9]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.2, random_state=20)
print('20% Holdout:', str(regr1.fit(X_train1, y_train1).score(X_train1, y_train1)))
print('Test on Sample:', str(regr1.fit(X1, y1).score(X1, y1)))

20% Holdout: 0.8472102359190959
Test on Sample: 0.8658812863422006


In [10]:
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=20)
print('20% Holdout:', str(regr2.fit(X_train2, y_train2).score(X_train2, y_train2)))
print('Test on Sample:', str(regr2.fit(X2, y2).score(X2, y2)))

20% Holdout: 0.8712164708676707
Test on Sample: 0.8662173350867853


In [11]:
cv_scores1 = cross_val_score(regr1, X1, y1, cv=4)
cv_r21 = cross_val_score(regr1, X1, y1, cv=4, scoring='r2')
print("Coefficients:", regr1.coef_)
print("\nCV:", cv_scores1)
print("\nR Squared:", cv_r21, "\nMean:", cv_r21.mean(), "\nStd:", cv_r21.std())

Coefficients: [[ 3.44157767e-02 -2.23888124e-10  1.43562879e+01 -9.44786464e+01]]

CV: [ 8.25565728e-01  4.77427616e-01 -2.82479184e+05  7.25545201e-01]

R Squared: [ 8.25565728e-01  4.77427616e-01 -2.82479184e+05  7.25545201e-01] 
Mean: -70619.2889146158 
Std: 122317.36757184095


In [12]:
cv_scores2 = cross_val_score(regr2, X2, y2, cv=4)
cv_r22 = cross_val_score(regr2, X2, y2, cv=4, scoring='r2')
print("Coefficients:", regr2.coef_)
print("\nCV:", cv_scores2)
print("\nR Squared:", cv_r22, "\nMean:", cv_r22.mean(), "\nStd:", cv_r22.std())

Coefficients: [[ 3.24644697e-02 -1.95120694e-10  9.62487082e+00 -1.18016945e+02]]

CV: [ 7.97290750e-01 -5.94719293e-01 -3.49427245e+05  6.66908099e-01]

R Squared: [ 7.97290750e-01 -5.94719293e-01 -3.49427245e+05  6.66908099e-01] 
Mean: -87356.59382010742 
Std: 151306.5608678155
